[IPython Notebook](legacy.ipynb) |  [Python Script](legacy.py)

Comparing PHOEBE 2.0 vs PHOEBE Legacy
============================

**NOTE**: PHOEBE 1.0 legacy is an alternate backend and is not installed with PHOEBE 2.0.  In order to run this backend, you'll need to have [PHOEBE 1.0](https://phoebe-project.org/1.0) installed.

Setup
-----------------------------

As always, let's do imports and initialize a logger and a new bundle.  See [Building a System](../tutorials/building_a_system.html) for more details.

In [1]:
%matplotlib inline

In [2]:
import phoebe
from phoebe import u
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger()

b = phoebe.default_binary()
b['q'] = 0.7

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/astropy/units/quantity.py:732: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


Adding Datasets and Compute Options
--------------------

In [3]:
b.add_dataset('lc', times=np.linspace(0,10,101), dataset='lc01')
b.add_dataset('rv', times=np.linspace(0,10,101), dataset='rvdyn')
b.add_dataset('rv', times=np.linspace(0,10,101), dataset='rvnum')

<ParameterSet: 15 parameters | contexts: compute, dataset>

Let's add compute options for phoebe using both the new (marching) method for creating meshes as well as the WD method which imitates the format of the mesh used within legacy.

In [4]:
b.add_compute(compute='phoebe2marching', mesh_method='marching')

<ParameterSet: 32 parameters | datasets: lc01, rvdyn, rvnum>

In [5]:
b.add_compute(compute='phoebe2wd', mesh_method='wd', eclipse_method='graham')

ValueError: value must be one of ['marching']

Now we add compute options for the 'legacy' backend.

In [ ]:
b.add_compute('legacy', compute='phoebe1')

And set the two RV datasets to use the correct methods (for both compute options)

In [ ]:
b.set_value_all('rv_method', dataset='rvdyn', value='dynamical')

In [ ]:
b.set_value_all('rv_method', dataset='rvnum', value='flux-weighted')

The 'blackbody' atmospheres from phoebe1 are comperable to the 'extern_planckint' atmospheres in phoebe.

In [ ]:
b.set_value_all('atm@phoebe2wd', 'extern_planckint')
b.set_value_all('atm@phoebe2marching', 'extern_planckint')
b.set_value_all('atm@phoebe1', 'blackbody')

Let's make sure both 'phoebe1' and 'phoebe2wd' use the same value for gridsize

In [ ]:
b.set_value_all('gridsize', 30)

Let's also disable other special effect such as heating, gravity, and light-time effects.

In [ ]:
b.set_value_all('ld_func', 'logarithmic')
b.set_value_all('ld_coeffs', [0.,0.])
# TODO: remove this once tested to work correctly with limbdarkening

In [ ]:
b.set_value_all('irrad_method', 'none') # phoebe
b.set_value('refl_num', 0) # legacy

In [ ]:
b.set_value_all('rv_grav', False)
# TODO: phoebe1???
# TODO: remove these?

In [ ]:
b.set_value_all('ltte', False)

Finally, let's compute all of our models

In [ ]:
b.run_compute(compute='phoebe2marching', model='phoebe2marchingmodel')

In [ ]:
b.run_compute(compute='phoebe2wd', model='phoebe2wdmodel')

In [ ]:
b.run_compute(compute='phoebe1', model='phoebe1model')

Plotting
-------------------------

### Light Curve

In [ ]:
axs, artists = b['lc01@phoebe2marchingmodel'].plot(color='g')
axs, artists = b['lc01@phoebe2wdmodel'].plot(color='b')
axs, artists = b['lc01@phoebe1model'].plot(color='r')
leg = plt.legend(loc=4)

Now let's plot the residuals between these two models

In [ ]:
artist, = plt.plot(b.get_value('fluxes@lc01@phoebe2marchingmodel') - b.get_value('fluxes@lc01@phoebe1model'), 'g-')
artist, = plt.plot(b.get_value('fluxes@lc01@phoebe2wdmodel') - b.get_value('fluxes@lc01@phoebe1model'), 'b-')
artist = plt.axhline(0.0, linestyle='dashed', color='k')

### Dynamical RVs

Since the dynamical RVs don't depend on the mesh, there should be no difference between the 'phoebe2marching' and 'phoebe2wd' synthetic models.  Here we'll just choose one to plot.

In [ ]:
axs, artists = b['rvdyn@phoebe2wdmodel'].plot(color='b')
axs, artists = b['rvdyn@phoebe1model'].plot(color='r')

And also plot the residuals of both the primary and secondary RVs (notice the scale on the y-axis)

In [ ]:
artist, = plt.plot(b.get_value('rvs@rvdyn@primary@phoebe2wdmodel') - b.get_value('rvs@rvdyn@primary@phoebe1model'), color='b', ls=':')
artist, = plt.plot(b.get_value('rvs@rvdyn@secondary@phoebe2wdmodel') - b.get_value('rvs@rvdyn@secondary@phoebe1model'), color='b', ls='-.')
artist = plt.axhline(0.0, linestyle='dashed', color='k')
ylim = plt.ylim(-1e-12, 1e-12)

### Numerical (flux-weighted) RVs

In [ ]:
axs, artists = b['rvnum@phoebe2marchingmodel'].plot(color='g')
axs, artists = b['rvnum@phoebe2wdmodel'].plot(color='b')
axs, artists = b['rvnum@phoebe1model'].plot(color='r')

In [ ]:
artist, = plt.plot(b.get_value('rvs@rvnum@primary@phoebe2marchingmodel', ) - b.get_value('rvs@rvnum@primary@phoebe1model'), color='g', ls=':')
artist, = plt.plot(b.get_value('rvs@rvnum@secondary@phoebe2marchingmodel') - b.get_value('rvs@rvnum@secondary@phoebe1model'), color='g', ls='-.')

artist, = plt.plot(b.get_value('rvs@rvnum@primary@phoebe2wdmodel', ) - b.get_value('rvs@rvnum@primary@phoebe1model'), color='b', ls=':')
artist, = plt.plot(b.get_value('rvs@rvnum@secondary@phoebe2wdmodel') - b.get_value('rvs@rvnum@secondary@phoebe1model'), color='b', ls='-.')

artist = plt.axhline(0.0, linestyle='dashed', color='k')
ylim = plt.ylim(-1e-2, 1e-2)